In [1]:
import tensorflow as tf

# tensorflowjs_converter --input_format=tfjs_layers_model --output_format=keras_saved_model ./work/data/tfjs_models/model_4k/model_4k.json ./work/data/keras_models/model_4k/

# load model from tfjs
model = tf.keras.models.load_model('./work/data/keras_models/model_4k/')

2022-05-07 21:07:29.549038: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-07 21:07:29.549060: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-07 21:07:29.549074: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (devitra): /proc/driver/nvidia/version does not exist
2022-05-07 21:07:29.549188: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [57]:
import requests

def load_embedding(embedding_name: str, retained_size: int) -> list:
    url = f'http://localhost:3080/embedding?embedding={embedding_name}'
    return requests.get(url).json()['embedding'][:retained_size]

embedding = load_embedding('pymde', 4000)

def scale_rating(rating: int) -> float:
    if rating > 5:
        return 1.0
    else:
        return -1.0

class AnimelistItem:
    def __init__(self, anime_ix: int, rating: int):
        self.anime_ix = anime_ix
        self.rating = rating

    def title(self):
        return embedding[self.anime_ix]['metadata']['title']

    def __str__(self):
        return f'{self.title()}: {self.rating}'

    def __repr__(self):
        return self.__str__()

def load_user_animelist(username: str) -> list:
    url = 'http://localhost:3080/recommendation/training/trainingData'
    payload = [username]
    res = requests.post(url, json=payload).json()['trainingData'][0]
    # print(url, res)
    return [AnimelistItem(rating['animeIx'], rating['rating']) for rating in res if rating.get('rating') is not None and rating.get('animeIx') is not None and rating['animeIx'] < len(embedding)]

def build_model_input(animelist: list) -> list:
    input_arr = [0] * len(embedding)
    for ranking in animelist:
        if ranking.anime_ix < len(embedding):
            input_arr[ranking.anime_ix] = scale_rating(ranking.rating)
    input_tensor = tf.constant([input_arr], dtype=tf.float32)
    return input_tensor

In [55]:
def get_recommendations(username: str, num_recommendations: int) -> list:
    animelist = [] if username is None else load_user_animelist(username)
    # print(animelist)
    input_tensor = build_model_input(animelist)
    prediction = model(input_tensor).numpy()[0].tolist()
    
    # sort by prediction
    sorted_predictions = sorted(enumerate(prediction), key=lambda x: x[1], reverse=True)
    # filter out already watched anime
    filtered_predictions = [x for x in sorted_predictions if x[0] not in [ranking.anime_ix for ranking in animelist]]
    return [AnimelistItem(ix, pred) for ix, pred in filtered_predictions[:num_recommendations]]

In [68]:
get_recommendations('ameo___', 50)

[Nisemonogatari: 0.975067675113678,
 Owarimonogatari 2nd Season: 0.8810790181159973,
 Zoku Owarimonogatari: 0.7453194856643677,
 Koe no Katachi: 0.5092713832855225,
 New Game!: 0.4990721344947815,
 Kobayashi-san Chi no Maid Dragon: 0.4669526517391205,
 Cowboy Bebop: 0.40972402691841125,
 Mononoke Hime: 0.39753350615501404,
 FLCL: 0.3481951355934143,
 Lucky☆Star: Original na Visual to Animation: 0.3351402282714844,
 Mahou Shoujo Madoka★Magica: 0.32533931732177734,
 Howl no Ugoku Shiro: 0.3230133652687073,
 Tonari no Totoro: 0.3213270306587219,
 Non Non Biyori: 0.30446016788482666,
 K-On!: 0.29454854130744934,
 Yuru Camp△: 0.2840392589569092,
 One Punch Man: 0.28098371624946594,
 Neon Genesis Evangelion: 0.2763269245624542,
 Kotonoha no Niwa: 0.27423644065856934,
 Steins;Gate 0: 0.2643921375274658,
 Acchi Kocchi: Place=Princess: 0.26396670937538147,
 Made in Abyss: 0.26230040192604065,
 Paprika: 0.2542864680290222,
 Perfect Blue: 0.2527683675289154,
 K-On!!: 0.2513952851295471,
 Another: